<a href="https://colab.research.google.com/github/muharyaanandas/CNN-Waste-Classification/blob/main/MobileNet_Root%20Mean%20Square%20Propagation%20(RMSprop).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
#Read all image types

def list_images(basePath, contains=None):
    # return the set of files that are valid
    return list_files(basePath, validExts=(".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"), contains=contains)

def list_files(basePath, validExts=(".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"), contains=None):
    # loop over the directory structure
    for (rootDir, dirNames, filenames) in os.walk(basePath):
        # loop over the filenames in the current directory
        for filename in filenames:
            # if the contains string is not none and the filename does not contain
            # the supplied string, then ignore the file
            if contains is not None and filename.find(contains) == -1:
                continue

            # determine the file extension of the current file
            ext = filename[filename.rfind("."):].lower()

            # check to see if the file is an image and should be processed
            if ext.endswith(validExts):
                # construct the path to the image and yield it
                imagePath = os.path.join(rootDir, filename).replace(" ", "\\ ")
                yield imagePath

In [ ]:
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer

In [ ]:
# Define data path
data_dir = '/content/drive/MyDrive/Klasifikasi_Sampah/Dataset'
dirs = os.listdir(data_dir)

# Initialize a dictionary to store image counts
image_counts = {}

# Load images and labels
data_list = []
labels = []
class_label = 0
for dir_name in dirs:
    path = os.path.join(data_dir, dir_name)
    image_files = glob.glob(os.path.join(path, '*'))
    image_count = 0
    for image_file in image_files:
        img = cv2.imread(image_file)
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (224, 224))
            data_list.append(img)
            labels.append(class_label)
            image_count += 1
    image_counts[dir_name] = image_count  # Store the count of images per category
    class_label += 1

# Print the total images for each category
for category, count in image_counts.items():
    print(f"Total {category} images: {count}")

data_array = np.array(data_list) / 255.0
labels_array = np.array(labels)

Total Organik images: 100
Total Anorganik images: 109
Total B3 images: 101


In [ ]:
# Encode labels
label_binarizer = LabelBinarizer()
encoded_labels = label_binarizer.fit_transform(labels_array)

# Splitting dataset
X_train, X_test, Y_train, Y_test = train_test_split(data_array, encoded_labels, test_size=0.2, stratify=encoded_labels, random_state=24)

# Augmentasi data
datagen = ImageDataGenerator(
    rotation_range=30,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)
datagen.fit(X_train)

In [ ]:
# Model MobileNet
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = True  # Unfreeze the base model for fine-tuning

# Membangun model di atas MobileNet
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

17225924/17225924 [==============================] - 0s 0us/step


In [ ]:
# Kompilasi model dengan optimizer RMSprop
model.compile(
    optimizer=RMSprop(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Callbacks
checkpoint_filepath = '/content/drive/MyDrive/Klasifikasi_Sampah/MobileNet/Epoch_10_RMSprop/Model2.hdf5'
save_model = ModelCheckpoint(filepath=checkpoint_filepath, save_best_only=True, monitor='val_accuracy', mode='max')
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)

# Melatih model
history = model.fit(
    datagen.flow(X_train, Y_train, batch_size=32),
    validation_data=(X_test, Y_test),
    epochs=10,
    callbacks=[save_model, early_stopping, reduce_lr]
)

Epoch 1/10
8/8 [==============================] - ETA: 0s - loss: 1.4455 - accuracy: 0.4113

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - 61s 7s/step - loss: 1.4455 - accuracy: 0.4113 - val_loss: 1.0872 - val_accuracy: 0.4032 - lr: 1.0000e-04
Epoch 2/10
8/8 [==============================] - 54s 7s/step - loss: 1.0141 - accuracy: 0.5444 - val_loss: 1.0114 - val_accuracy: 0.4677 - lr: 1.0000e-04
Epoch 3/10
8/8 [==============================] - 53s 6s/step - loss: 0.8921 - accuracy: 0.6008 - val_loss: 0.8934 - val_accuracy: 0.5000 - lr: 1.0000e-04
Epoch 4/10
8/8 [==============================] - 55s 7s/step - loss: 0.7377 - accuracy: 0.6694 - val_loss: 0.8933 - val_accuracy: 0.5484 - lr: 1.0000e-04
Epoch 5/10
8/8 [==============================] - 58s 7s/step - loss: 0.7199 - accuracy: 0.6935 - val_loss: 0.9159 - val_accuracy: 0.5484 - lr: 1.0000e-04
Epoch 6/10
8/8 [==============================] - 54s 7s/step - loss: 0.5796 - accuracy: 0.7339 - val_loss: 0.7933 - val_accuracy: 0.6452 - lr: 1.0000e-04
Epoch 7/10
8/8 [==============================] - 56s 7s/step - loss: 0.5626 - ac

In [ ]:
# Evaluasi model
best_model = tf.keras.models.load_model(checkpoint_filepath)
pred_test = best_model.predict(X_test)
pred_test_labels = np.argmax(pred_test, axis=1)
y_test_labels = np.argmax(Y_test, axis=1)

print(classification_report(y_test_labels, pred_test_labels))
print("Accuracy on test set: ", accuracy_score(y_test_labels, pred_test_labels))

2/2 [==============================] - 3s 983ms/step
              precision    recall  f1-score   support

           0       0.88      0.70      0.78        20
           1       1.00      0.50      0.67        22
           2       0.54      0.95      0.69        20

    accuracy                           0.71        62
   macro avg       0.81      0.72      0.71        62
weighted avg       0.81      0.71      0.71        62

Accuracy on test set:  0.7096774193548387


In [ ]:
# Convert the model to TFLite format
converter = tf.lite.TFLiteConverter.from_keras_model(best_model)
tflite_model = converter.convert()

# Save the TFLite model to file
tflite_model_filepath = '/content/drive/MyDrive/Klasifikasi_Sampah/MobileNet/Epoch_10_RMSprop/model.tflite'
with open(tflite_model_filepath, 'wb') as file:
    file.write(tflite_model)

print(f"Model saved as .hdf5 at {checkpoint_filepath}")
print(f"Model saved as .tflite at {tflite_model_filepath}")

Model saved as .hdf5 at /content/drive/MyDrive/Klasifikasi_Sampah/MobileNet/Epoch_10_RMSprop/Model2.hdf5
Model saved as .tflite at /content/drive/MyDrive/Klasifikasi_Sampah/MobileNet/Epoch_10_RMSprop/model.tflite
